In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# PostgreSQL to BigQuery Migration
<table align="left">
<td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/dataproc-templates/blob/main/notebooks/postgresql2bq/postgresql-to-bigquery-notebook.ipynb">
        <img src="../images/colab-logo-32px.png" alt="Colab logo" />Run in Colab
    </a>
</td>
<td>
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fdataproc-templates%2Fmain%2Fnotebooks%2Fpostgresql2bq%2Fpostgresql-to-bigquery-notebook.ipynb">
        <img src="../images/colab-enterprise-logo-32px.png" alt="GCP Colab Enterprise logo" />Run in Colab Enterprise
    </a>
</td>
<td>
    <a href="https://github.com/GoogleCloudPlatform/dataproc-templates/blob/main/notebooks/postgresql2bq/postgresql-to-bigquery-notebook.ipynb">
        <img src="../images/github-logo-32px.png" alt="GitHub logo" />View on GitHub
    </a>
</td>
<td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/dataproc-templates/main/notebooks/postgresql2bq/postgresql-to-bigquery-notebook.ipynb">
        <img src="../images/vertexai.jpg" alt="Vertex AI logo" />Open in Vertex AI Workbench
    </a>
</td>
</table>

## References
* [DataprocPySparkBatchOp reference](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.0/google_cloud_pipeline_components.experimental.dataproc.html) 
* [Kubeflow SDK Overview](https://www.kubeflow.org/docs/components/pipelines/sdk/sdk-overview/)
* [Dataproc Serverless in Vertex AI Pipelines tutorial](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage3/get_started_with_dataproc_serverless_pipeline_components.ipynb)
* [Build a Vertex AI Pipeline](https://cloud.google.com/vertex-ai/docs/pipelines/build-pipeline)

This notebook is built to run a Vertex AI User-Managed Notebook using the default Compute Engine Service Account. Check the Dataproc Serverless in Vertex AI Pipelines tutorial linked above to learn how to setup a different Service Account.

## Permissions
Make sure that the service account used to run the notebook has the following roles:

* roles/aiplatform.serviceAgent
* roles/aiplatform.customCodeServiceAgent
* roles/storage.objectCreator
* roles/storage.objectViewer
* roles/dataproc.editor
* roles/dataproc.worker

# Step 1: Install Libraries
<div class="alert alert-block alert-info">
<b>NOTE: </b>Run Step 1 one time for each new notebook instance</div

In [ ]:
!pip3 install SQLAlchemy
!pip3 install --upgrade google-cloud-pipeline-components kfp --user -q
!pip3 install psycopg2-binary

In [ ]:
!sudo apt-get update -y
!sudo apt-get install default-jdk -y

### Once you've installed the additional packages, you may need to restart the notebook kernel so it can find the packages.

Uncomment & Run this cell if you have installed anything from above commands

In [ ]:
# import os
# import IPython
# if not os.getenv("IS_TESTING"):
#     app = IPython.Application.instance()
#     app.kernel.do_shutdown(True)

# Step 2: Import Libraries

In [ ]:
import google.cloud.aiplatform as aiplatform
import sys, os
from kfp import dsl
from kfp import compiler
from datetime import datetime
import time
import copy
import json
import pandas as pd

try:
    from google_cloud_pipeline_components.experimental.dataproc import DataprocPySparkBatchOp
except ModuleNotFoundError:
    from google_cloud_pipeline_components.v1.dataproc import DataprocPySparkBatchOp
    
import sqlalchemy
from sqlalchemy import text
import psycopg2
import math
from pathlib import Path

# Step 3: Assign Parameters

## Step 3.1 Common Parameters

GCP_PROJECT : GCP project-id

REGION : GCP region

GCS_STAGING_LOCATION : Cloud Storage staging location to be used for this notebook to store artifacts

SUBNET : VPC subnet

JARS : list of jars. For this notebook postgre connectora and postgres connectorjar is required in addition with the dataproc template

MAX_PARALLELISM : Parameter for number of jobs to run in parallel default value is 5

SERVICE_ACCOUNT: Custom service account email to use for vertex ai pipeline and dataproc job with above mentioned permissions

POSTGRESQL_TO_BIGQUERY_JOBS : List of bigquery job IDs that will be created by Vertex AI pipelines to migrate data from source to BQ.

In [ ]:
IS_PARAMETERIZED = False

In [ ]:
if not IS_PARAMETERIZED:
    GCP_PROJECT = ""
    REGION = "" # eg: us-central1 (any valid GCP region)
    GCS_STAGING_LOCATION = "" # eg: gs://my-staging-bucket/sub-folder
    SUBNET = "projects/{project}/regions/{region}/subnetworks/{subnet}"
    MAX_PARALLELISM = 5 # max number of tables which will migrated parallelly 
    SERVICE_ACCOUNT = "" 
    
# Do not change this parameter unless you want to refer below JARS from new location
JARS = [GCS_STAGING_LOCATION + "/jars/postgresql-42.2.6.jar", GCS_STAGING_LOCATION + "/jars/spark-bigquery-with-dependencies_2.12-0.27.0.jar"]


In [ ]:

POSTGRESQL_TO_BIGQUERY_JOBS = []
# If SERVICE_ACCOUNT is not specified it will take the one attached to Notebook
if SERVICE_ACCOUNT == '':
    shell_output = !gcloud auth list 2>/dev/null
    SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()
    print("Service Account: ",SERVICE_ACCOUNT)
    


## Step 3.2 PostgreSQL Parameters

POSTGRES_HOST : PostgreSQL instance ip address

POSTGRES_PORT : PostgreSQL instance port

POSTGRES_USERNAME : PostgreSQL username

POSTGRES_PASSWORD : PostgreSQL password

POSTGRES_DATABASE : name of database that you want to migrate

POSTGRESQL_TABLE_LIST: leave list empty for migrating complete database else provide tables as ['dbo.table1','sys.table2']. If this parameter is not empty, leave POSTGRESQL_SCHEMA_LIST empty.

POSTGRESQL_SCHEMA_LIST: leave list empty for migrating complete database else provide schema as ['schema1','schema2'] for migrating all tables in specific schemas. If this parameter is not empty, leave POSTGRESQL_TABLE_LIST empty.

BIGQUERY_DATASET: name of the dataset to migrate

BIGQUERY_MODE:  output write mode (Default: overwrite)

In [ ]:
if not IS_PARAMETERIZED:
    POSTGRESQL_HOST = ""
    POSTGRESQL_PORT = ""
    POSTGRESQL_USERNAME = ""
    POSTGRESQL_PASSWORD = ""
    POSTGRESQL_DATABASE = ""
    POSTGRESQL_TABLE_LIST = [] # leave list empty for migrating complete database else provide tables as ['table1','table2']
    POSTGRESQL_SCHEMA_LIST = [] # leave list empty for migrating complete database else provide schema as ['schema1','schema2'] for migrating all tables in specific schemas. If this parameter is not empty, leave POSTGRESQL_TABLE_LIST empty.

    BIGQUERY_DATASET = ""
    BIGQUERY_MODE = "" 

# Step 3.3 Notebook Configuration Parameters

<div class="alert alert-block alert-warning">
<b>NOTE: </b>Below variables should not be changed unless required</div>


In [ ]:
cur_path = Path(os.getcwd())

if IS_PARAMETERIZED:
    WORKING_DIRECTORY = os.path.join(cur_path.parent ,'python')
else:
    WORKING_DIRECTORY = os.path.join(cur_path.parent.parent ,'python')

# If the above code doesn't fetches the correct path please
# provide complete path to python folder in your dataproc 
# template repo which you cloned 

# WORKING_DIRECTORY = "/home/jupyter/dataproc-templates/python/"
print(WORKING_DIRECTORY)

In [ ]:
PY_JDBC_DRIVER = "postgresql+psycopg2"
JDBC_DRIVER = "org.postgresql.Driver"
JDBC_URL="jdbc:postgresql://{0}:{1}/{2}?user={3}&password={4}&reWriteBatchedInserts=True".format(POSTGRESQL_HOST,POSTGRESQL_PORT,POSTGRESQL_DATABASE,POSTGRESQL_USERNAME,POSTGRESQL_PASSWORD)
MAIN_CLASS = "com.google.cloud.dataproc.templates.main.DataProcTemplate"
PACKAGE_EGG_FILE = "dataproc_templates_distribution.egg"
PIPELINE_ROOT = GCS_STAGING_LOCATION + "/pipeline_root/dataproc_pyspark"

# Step 4: Generate PostgreSQL Table List

This step creates list of tables for migration. 

* If POSTGRESQL_TABLE_LIST and POSTGRESQL_SCHEMA_LIST are kept empty, then all the tables in the POSTGRESQL_DATABASE are listed for migration.

* If POSTGRESQL_SCHEMA_LIST is non empty, then all tables associated with the mentioned schemas will be listed for migration

* If POSTGRESQL_TABLE_LIST is non empty, then the provided list of tables are selected for migration

In [ ]:
# removing defaut value if the list is empty
if len(POSTGRESQL_SCHEMA_LIST) == 1 and POSTGRESQL_SCHEMA_LIST[0] == '':
    POSTGRESQL_SCHEMA_LIST.pop()


if len(POSTGRESQL_TABLE_LIST) == 1 and POSTGRESQL_TABLE_LIST[0] == '':
    POSTGRESQL_TABLE_LIST.pop()
    
if POSTGRESQL_SCHEMA_LIST and POSTGRESQL_TABLE_LIST:
    print(POSTGRESQL_SCHEMA_LIST , POSTGRESQL_TABLE_LIST)
    print(POSTGRESQL_SCHEMA_LIST and POSTGRESQL_TABLE_LIST)
    sys.exit("Please provide values for either POSTGRESQL_SCHEMA_LIST OR POSTGRESQL_TABLE_LIST. Non empty values for both the values at the same time are not accepted")

In [ ]:
DB = sqlalchemy.create_engine(
            sqlalchemy.engine.url.URL.create(
                drivername=PY_JDBC_DRIVER,
                username=POSTGRESQL_USERNAME,
                password=POSTGRESQL_PASSWORD,
                database=POSTGRESQL_DATABASE,
                host=POSTGRESQL_HOST,
                port=POSTGRESQL_PORT
              )
            )

with DB.connect() as conn:
        print("connected to database")
        if not POSTGRESQL_TABLE_LIST and not POSTGRESQL_SCHEMA_LIST: # Migrate all possible tables from database
            query_str = "select TABLE_SCHEMA,TABLE_NAME from INFORMATION_SCHEMA.Tables where TABLE_SCHEMA not in ('pg_catalog','information_schema');"
            results = conn.execute(text(query_str)).fetchall()

        elif POSTGRESQL_SCHEMA_LIST and not POSTGRESQL_TABLE_LIST: # Only Migrate tables associated with the provided schema list
            results = conn.execute(text("select TABLE_SCHEMA,TABLE_NAME from INFORMATION_SCHEMA.Tables where TABLE_SCHEMA in ('{}');".format("','".join(POSTGRESQL_SCHEMA_LIST)))).fetchall()

        # when POSTGRESQL_TABLE_LIST is already not empty, only mentioned tables will be migrated

        print("Total Tables = ", len(results))
        for row in results:
            POSTGRESQL_TABLE_LIST.append(row[0]+"."+row[1])

        print("list of tables for migration :")
        print(POSTGRESQL_TABLE_LIST)


        

# Step 5: Get Primary Keys for partitioning the tables

This step fetches primary key from POSTGRESQL_DATABASE for the tables listed for migration

In [ ]:
with DB.connect() as conn:

    POSTGRESQL_TABLE_PRIMARY_KEYS = {}
    for table in POSTGRESQL_TABLE_LIST:
            primary_keys = []
            results = conn.execute(text("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS T JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE K ON K.CONSTRAINT_NAME=T.CONSTRAINT_NAME  WHERE  K.TABLE_NAME='{0}'  AND K.TABLE_SCHEMA='{1}' AND T.CONSTRAINT_TYPE='PRIMARY KEY';".format(table.split(".")[1],table.split(".")[0]))).fetchall()
            # print(results)
            for row in results:
                primary_keys.append(row[0])
            if primary_keys:
                POSTGRESQL_TABLE_PRIMARY_KEYS[table] = ",".join(primary_keys)
            else:
                POSTGRESQL_TABLE_PRIMARY_KEYS[table] = ""



In [ ]:
pkDF = pd.DataFrame({"table" : POSTGRESQL_TABLE_LIST, "primary_keys": list(POSTGRESQL_TABLE_PRIMARY_KEYS.values())})
print("Below are identified primary keys for migrating postgresql table to bigquery:")
pkDF

# Step 6: Create JAR files and Upload to Cloud Storage
<div class="alert alert-block alert-info">
<b>NOTE: </b> Run Step 6 one time for each new notebook instance</div>

In [ ]:
%cd $WORKING_DIRECTORY

### Get JDBC Connector jars

In [ ]:
%%bash

wget https://jdbc.postgresql.org/download/postgresql-42.2.6.jar

wget https://repo1.maven.org/maven2/com/google/cloud/spark/spark-bigquery-with-dependencies_2.12/0.27.0/spark-bigquery-with-dependencies_2.12-0.27.0.jar

### Build Dataproc Templates python package

In [ ]:
! python ./setup.py bdist_egg --output=$PACKAGE_EGG_FILE

### Copying JAR files to GCS_STAGING_LOCATION

In [ ]:
! gcloud storage cp main.py $GCS_STAGING_LOCATION/
! gcloud storage cp --recursive $PACKAGE_EGG_FILE $GCS_STAGING_LOCATION/
! gcloud storage cp postgresql-42.2.6.jar $GCS_STAGING_LOCATION/jars/postgresql-42.2.6.jar
! gcloud storage cp spark-bigquery-with-dependencies_2.12-0.27.0.jar $GCS_STAGING_LOCATION/jars/spark-bigquery-with-dependencies_2.12-0.27.0.jar

# Step 7: Calculate Parallel Jobs for POSTGRESQL to BigQuery

This step uses MAX_PARALLELISM parameter to calculate number of parallel jobs to run

In [ ]:
# calculate parallel jobs:
COMPLETE_LIST = copy.deepcopy(POSTGRESQL_TABLE_LIST)
PARALLEL_JOBS = len(POSTGRESQL_TABLE_LIST)//MAX_PARALLELISM
JOB_LIST = []
while len(COMPLETE_LIST) > 0:
    SUB_LIST = []
    for i in range(MAX_PARALLELISM):
        if len(COMPLETE_LIST)>0 :
            SUB_LIST.append(COMPLETE_LIST[0])
            COMPLETE_LIST.pop(0)
        else:
            break
    JOB_LIST.append(SUB_LIST)
print("list of tables for execution : ")
print(JOB_LIST)

# Step 8: Get Row Count of Tables and identify Partition Columns

This step uses PARTITION_THRESHOLD (default value is 1 million) parameter and any table having rows greater than PARTITION_THRESHOLD will be partitioned based on Primary Keys
Get Primary keys for all tables to be migrated and find an integer column to partition on

In [ ]:
PARTITION_THRESHOLD = 1000000
CHECK_PARTITION_COLUMN_LIST={}

In [ ]:
with DB.connect() as conn:
    for table in POSTGRESQL_TABLE_LIST:
        results = conn.execute(text("SELECT count(1) FROM {}".format(table))).fetchall()
        # print(results)
        if results[0][0]>PARTITION_THRESHOLD and len(POSTGRESQL_TABLE_PRIMARY_KEYS.get(table).split(",")[0])>0:
            column_list=POSTGRESQL_TABLE_PRIMARY_KEYS.get(table).split(",")
            for column in column_list:
                results_datatype = DB.execute("SELECT DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = '{0}' AND TABLE_NAME   = '{1}' AND COLUMN_NAME  = '{2}'".format(table.split(".")[0],table.split(".")[1],column)).fetchall()      
                # print(results_datatype)
                if results_datatype[0][0]=="int":
                    lowerbound = DB.execute("SELECT min({0}) from {1}".format(column,table)).fetchall()
                    upperbound = DB.execute("SELECT max({0}) from {1}".format(column,table)).fetchall()
                    numberPartitions = math.ceil((upperbound[0][0]-lowerbound[0][0])/PARTITION_THRESHOLD)
                    CHECK_PARTITION_COLUMN_LIST[table]=[column,lowerbound[0][0],upperbound[0][0],numberPartitions]
                    print(CHECK_PARTITION_COLUMN_LIST)


# Step 9: Execute Pipeline to Migrate tables from POSTGRESQL to BIGQUERY

* BIGQUERY_DATASET : Target dataset in Bigquery
* BIGQUERY_MODE : Mode of operation at target <append|overwrite|ignore|errorifexists> (default overwrite)
* TEMP_GCS_BUCKET : Bucket name for dataproc job staging
* PYTHON_FILE_URIS : Path to PACKAGE_EGG_FILE
* MAIN_PYTHON_CLASS : Path to main.py

In [ ]:
BIGQUERY_DATASET="python_dataproc_templates"
BIGQUERY_MODE = "overwrite"  # append/overwrite
TEMP_GCS_BUCKET="python-dataproc-templates-temp-bq"
PYTHON_FILE_URIS = [ GCS_STAGING_LOCATION + "/dataproc_templates_distribution.egg" ]
MAIN_PYTHON_CLASS = GCS_STAGING_LOCATION + "/main.py"

In [ ]:
def migrate_postgresql_to_bigquery(EXECUTION_LIST):
    EXECUTION_LIST = EXECUTION_LIST
    aiplatform.init(project=GCP_PROJECT,staging_bucket=TEMP_GCS_BUCKET)
    
    @dsl.pipeline(
        name="python-postgresql-to-bigquery-pyspark",
        description="Pipeline to get data from PostgreSQL to BigQuery",
    )
    def pipeline(
        PROJECT_ID: str = GCP_PROJECT,
        LOCATION: str = REGION,
        MAIN_PYTHON_CLASS: str = MAIN_PYTHON_CLASS,
        JAR_FILE_URIS: list = JARS,
        SUBNETWORK_URI: str = SUBNET,
        SERVICE_ACCOUNT: str = SERVICE_ACCOUNT,
        PYTHON_FILE_URIS: list = PYTHON_FILE_URIS
    ):
        for table in EXECUTION_LIST:
            BATCH_ID = "postgresql2bigquery-{}".format(datetime.now().strftime("%s"))
            POSTGRESQL_TO_BIGQUERY_JOBS.append(BATCH_ID)
            if table in CHECK_PARTITION_COLUMN_LIST.keys():
                TEMPLATE_SPARK_ARGS = [
                    "--template=JDBCTOBIGQUERY",
                    "--jdbc.bigquery.input.url={}".format(JDBC_URL),
                    "--jdbc.bigquery.input.driver={}".format(JDBC_DRIVER),
                    "--jdbc.bigquery.input.table={}".format(table),
                    "--jdbc.bigquery.output.mode={}".format(BIGQUERY_MODE),
                    "--jdbc.bigquery.output.table={}".format(table.split('.')[1]),
                    "--jdbc.bigquery.temp.bucket.name={}".format(TEMP_GCS_BUCKET),
                    "--jdbc.bigquery.output.dataset={}".format(BIGQUERY_DATASET),
                    "--jdbc.bigquery.input.partitioncolumn={}".format(CHECK_PARTITION_COLUMN_LIST[table][0]),
                    "--jdbc.bigquery.input.lowerbound={}".format(CHECK_PARTITION_COLUMN_LIST[table][1]),
                    "--jdbc.bigquery.input.upperbound={}".format(CHECK_PARTITION_COLUMN_LIST[table][2]),
                    "--jdbc.bigquery.numpartitions={}".format(CHECK_PARTITION_COLUMN_LIST[table][3])
                ]
            else:
                TEMPLATE_SPARK_ARGS = [
                    "--template=JDBCTOBIGQUERY",
                    "--jdbc.bigquery.input.url={}".format(JDBC_URL),
                    "--jdbc.bigquery.input.driver={}".format(JDBC_DRIVER),
                    "--jdbc.bigquery.input.table={}".format(table),
                    "--jdbc.bigquery.output.mode={}".format(BIGQUERY_MODE),
                    "--jdbc.bigquery.output.table={}".format(table.split('.')[1]),
                    "--jdbc.bigquery.temp.bucket.name={}".format(TEMP_GCS_BUCKET),
                    "--jdbc.bigquery.output.dataset={}".format(BIGQUERY_DATASET)
                ]

            _ = DataprocPySparkBatchOp(
                project=PROJECT_ID,
                location=LOCATION,
                batch_id=BATCH_ID,
                main_python_file_uri=MAIN_PYTHON_CLASS,
                jar_file_uris=JAR_FILE_URIS,
                python_file_uris=PYTHON_FILE_URIS,
                subnetwork_uri=SUBNETWORK_URI,
                service_account=SERVICE_ACCOUNT,
                runtime_config_version="1.1", # issue 665
                args=TEMPLATE_SPARK_ARGS
                )
            time.sleep(3)

    compiler.Compiler().compile(pipeline_func=pipeline, package_path="pipeline.json")

    pipeline = aiplatform.PipelineJob(
        display_name="pipeline",
        template_path="pipeline.json",
        pipeline_root=PIPELINE_ROOT,
        enable_caching=False,
        )
    pipeline.run(service_account=SERVICE_ACCOUNT)

In [ ]:
for execution_list in JOB_LIST:
    print(execution_list)
    migrate_postgresql_to_bigquery(execution_list)

# Step 10: Get status for tables migrated from PostgreSQL to BIGQUERY

In [ ]:
def get_bearer_token():
    
    try:
        #Defining Scope
        CREDENTIAL_SCOPES = ["https://www.googleapis.com/auth/cloud-platform"]

        #Assigning credentials and project value
        credentials, project_id = google.auth.default(scopes=CREDENTIAL_SCOPES)

        #Refreshing credentials data
        credentials.refresh(requests.Request())

        #Get refreshed token
        token = credentials.token
        if token:
            return (token,200)
        else:
            return "Bearer token not generated"
    except Exception as error:
        return ("Bearer token not generated. Error : {}".format(error),500)

In [ ]:
from google.auth.transport import requests
import google

token = get_bearer_token()
token = get_bearer_token()
if token[1] == 200:
    print("Bearer token generated")
else:
    print(token)

In [ ]:
import requests

postgresql_to_bigquery_status = []
job_status_url = "https://dataproc.googleapis.com/v1/projects/{}/locations/{}/batches/{}"
for job in POSTGRESQL_TO_BIGQUERY_JOBS:
    auth = "Bearer " + token[0]
    url = job_status_url.format(GCP_PROJECT,REGION,job)
    headers = {
      'Content-Type': 'application/json; charset=UTF-8',
      'Authorization': auth 
    }
    response = requests.get(url, headers=headers)
    postgresql_to_bigquery_status.append(response.json()['state'])

In [ ]:
statusDF = pd.DataFrame({"table" : POSTGRESQL_TABLE_LIST,"postgresql_to_bigquery_job" : POSTGRESQL_TO_BIGQUERY_JOBS, "postgresql_to_bigquery_status" : postgresql_to_bigquery_status})
statusDF

# Step 11: Validate row counts of migrated tables from SQL Server to BigQuery

In [ ]:
postgresql_row_count = []
bq_row_count = []

In [ ]:
# get postgresql table counts
DB = sqlalchemy.create_engine(
            sqlalchemy.engine.url.URL.create(
                drivername=PY_JDBC_DRIVER,
                username=POSTGRESQL_USERNAME,
                password=POSTGRESQL_PASSWORD,
                database=POSTGRESQL_DATABASE,
                host=POSTGRESQL_HOST,
                port=POSTGRESQL_PORT
              )
            )

with DB.connect() as conn:
    for table in POSTGRESQL_TABLE_LIST:
        results = conn.execute(text("select count(*) from {}".format(table))).fetchall()
        for row in results:
            postgresql_row_count.append(row[0])

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

for table in POSTGRESQL_TABLE_LIST:
    results = client.query("SELECT row_count FROM {}.__TABLES__ where table_id = '{}'".format(BIGQUERY_DATASET,table.split('.')[1]))
    for row in results:
        bq_row_count.append(row[0])

In [ ]:
statusDF['postgresql_row_count'] = postgresql_row_count 
statusDF['bq_row_count'] = bq_row_count 
statusDF